In [5]:
%%bash
pip -q install pydicom opencv-python scikit-image
pip -q install pyradiomics

wget -q http://www.inf.ufpr.br/lferrari/imagens_ihq.tar.gz
!tar -xf imagens_ihq.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 52.6 MB/s eta 0:00:00


In [7]:
import cv2 as cv
import numpy as np
import os
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from joblib import dump, load
import pydicom as dicom
import radiomics
from radiomics import featureextractor
import SimpleITK as sitk
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt

In [70]:
def cut_images(input_path, new_width, new_height, output_path=None):
    """
    Cut images into the desired size and save the output images

    Params:
    input_path = path to the original images
    output_path = path to save the cut images
    new_width = width of the cut images
    new_height = height of the cut images

    Return:
    None
    """



    images_data = {}
    classes = []
    patients = []

    # Browse input path
    for class_dir in os.listdir(input_path):
        class_path = os.path.join(input_path, class_dir)

        # If it is a directory
        if os.path.isdir(class_path):



            # Save image id
            image_id = 1

            # Go through images
            for image_file in os.listdir(class_path):
                image_path = os.path.join(class_path, image_file)

                # Save patient id
                patient = image_file.split("_")[0]
                patients.append(patient)

                image = cv.imread(image_path)

                # If image exists
                if image is not None:

                    # Save subimage id
                    sub_id = 1

                    for i in range(0, image.shape[0], new_height):
                        for j in range(0, image.shape[1], new_width):

                            # Cut image into subimage
                            sub_image = image[i:i+new_height, j:j+new_width]

                            # Image name identifier
                            image_name = f"{patient}-img{image_id}-{sub_id}"

                            # Append image and its label to the dictionary
                            images_data[image_name] = sub_image

                            # images_names.append(image_name)
                            # images.append(sub_image)

                            classes.append(int(class_dir))


                            # Write subimage if an output path was given
                            if output_path != None:
                                # Create dir if it doesn't exist
                                os.makedirs(os.path.join(output_path, class_dir), exist_ok=True)
                                # Output file path
                                output_file = f"{image_name}.png"
                                output_file = os.path.join(output_path, class_dir, output_file)
                                # Save subimage
                                cv.imwrite(output_file, sub_image)

                            sub_id += 1

                    image_id += 1

    return (images_data, patients, classes)

In [68]:
def divide_folds(images, patients, classes):
    """
    Divides a dataset into folds for stratified k-fold cross-validation.

    Params:
    images = list of all images
    patients = list with patient id for each image
    classes = list with class for each image

    Return:
    folds = list of tuples, each tuple is one folder
    """
    # Create a list of unique indexes for patients
    unique_patients = list(set(patients))

    # Shuffle the list of unique indexes
    random.shuffle(unique_patients)

    # Divide patients into groups
    n_folds = 4 # since it's not an exact division, there will be 5 folds
    fold_size = len(unique_patients) // n_folds
    patients_folds = [unique_patients[i:i+fold_size] for i in range(0, len(unique_patients), fold_size)]

    # List to save folds
    folds = []

    # Divide images into folds based on patients
    for i, patients_folds in enumerate(patients_folds):
        train_patients = [p for p in unique_patients if p not in patients_folds]
        test_patients = patients_folds

        train_indices = [i for i, patient in enumerate(patients) if patient in train_patients]
        test_indices = [i for i, patient in enumerate(patients) if patient in test_patients]

        train_images = [images[i] for i in train_indices]
        test_images = [images[i] for i in test_indices]
        train_classes = [classes[i] for i in train_indices]
        test_classes = [classes[i] for i in test_indices]

        folds.append((train_images, test_images, train_classes, test_classes))

    return folds


In [92]:
def apply_thresholds(imgs):
    """
    Apply Otsu's and Adaptative thresholds to images

    Params:
    imgs = list of raw images

    Return:
    imgs_otsu = Otsu's thresholded images
    imgs_adapt = Adaptative thresholded images
    """

    for key, value in images_data.items():

        gray = cv.cvtColor(value, cv.COLOR_BGR2GRAY)

        # Otsu's thresholding
        _, th1 = cv.threshold(gray, 100, 1, cv.THRESH_BINARY+cv.THRESH_OTSU)

        # Adaptative gaussian thresholding
        #th2 = cv.adaptiveThreshold(img,255,cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY,11,2)
        th3 = cv.adaptiveThreshold(gray, 1, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)

        images_data[key] = (value, th1, th3)

    return images_data

In [93]:
# Cut images into 40x30
images_data, patients, classes = cut_images("/content/imagens_ihq_er", 40, 30)

# Divide images into folds
images_names = list(images_data.keys())
folds = divide_folds(images_names, patients, classes)

# Update images_data, applying Otsu's and adaptive thresholds
images_data = apply_thresholds(images_data)

# Save folds locally
#dump(folds, '../folds.joblib')

# Load folds
# folds = load('../folds.joblib')

Extract features with PyRadiomics

In [ ]:
def run_extractor(imgs, otsu, adapt, extractor):
    """
    Extract features using sitk and pyradiomics

    Params:
    imgs = raw images
    otsu = masked images with otsu thresholding
    adapt = masked images with adaptative thresholding
    extractor = pyradiomics extractor

    Returns:
    features_otsu = features for otsu mask
    features_adapt = features for adaptative mask
    """

    data_spacing=[1,1,1]
    features_otsu = []
    features_adapt = []

    for idx in range(len(imgs)):

        # Get raw, Otsu's and adaptive images
        img = imgs[idx]
        img_otsu = otsu[idx]
        img_adapt = adapt[idx]

        sitk_img = sitk.GetImageFromArray(img)
        sitk_img.SetSpacing((1, 1, 1))
        sitk_img = sitk.JoinSeries(sitk_img)

        sitk_otsu = sitk.GetImageFromArray(img_otsu)
        sitk_otsu.SetSpacing((1, 1, 1))
        sitk_otsu = sitk.JoinSeries(sitk_otsu)
        sitk_otsu = sitk.Cast(sitk_otsu, sitk.sitkInt32)

        sitk_adapt = sitk.GetImageFromArray(img_adapt)
        sitk_adapt.SetSpacing((1, 1, 1))
        sitk_adapt = sitk.JoinSeries(sitk_adapt)
        sitk_adapt = sitk.Cast(sitk_otsu, sitk.sitkInt32)

        # Extract features and append them to the proper list
        try:
            ft_otsu = extractor.execute(sitk_img, sitk_otsu)
            features_otsu.append(ft_otsu)

            ft_adapt = extractor.execute(sitk_img, sitk_adapt)
            features_adapt.append(ft_adapt)

        except:
            #print(f"{idx}, ", end="")
            pass

    return (features_otsu, features_adapt)



In [101]:
def conditional_append(element, dest):
    """
    Append element to the list destiny, if element is not in destiny

    Params:
    element = an element of any kind
    dest = a destination list

    Returns:
    destiny = list with appended element if the element was not in there
    """
    if element not in dest:
        dest.append(element)

    return dest

def process_features(feats_o, feats_a):
    """
    Process features, in a way that:
    - features that are dictionaries and strings are removed
    - features that are tuples are separated and each element
    of the tuple is considered one feature
    - other types are converted to float

    Params:
    feats_o = list of Otsu's threshold features
    feats_a = list of adaptativa threshold features

    Returns:
    all_feats_o = Otsu's features processed
    all_feats_a = adaptative features processed
    names = feature names processed
    """

    all_feats_o = []
    all_feats_a = []
    names = []

    # For each image in one of the features list
    for sample_id in range(len(feats_o)):

        # Get features for Otsu's and adaptive for this sample
        sample_o = feats_o[sample_id]
        sample_a = feats_a[sample_id]

        values_o = []
        values_a = []

        # For each feature in the list
        for key in sample_o:

            # Get the feature's value
            value_o = sample_o[key]
            value_a = sample_a[key]

            # If the value is str or dict, ignore it
            if type(value_o) == str or type(value_o) == dict:
                continue
            # If it's a tuple
            elif type(value_o) == tuple:
                for e in range(len(value_o)):
                    # Add and index to the feature name
                    conditional_append(f'{key}_{e}', names)
                    # Append float values to the lists
                    values_o.append(float(value_o[e]))
                    values_a.append(float(value_a[e]))
            # For other data types, just append the name and float values
            else:
                conditional_append(key, names)
                values_o.append(float(value_o))
                values_a.append(float(value_a))

        # Append processed features to the general list
        all_feats_o.append(values_o)
        all_feats_a.append(values_a)

    return (all_feats_o, all_feats_a, names)

def extract_features(images_data):
    """
    Process features, in a way that:
    - features that are dictionaries and strings are removed
    - features that are tuples are separated and each element
    of the tuple is considered one feature
    - other types are converted to float
    Get the features' names, with tuple features indexed

    Params:
    folds = list of tuples containing x_train raw and with thresholds

    Returns:
    all_folds_feats = dictionary containing Otsu's features and adaptive
    features for each fold
    names = feature names
    """

    # Create feature extractor
    !wget -c https://raw.githubusercontent.com/AIM-Harvard/pyradiomics/master/examples/exampleSettings/Params.yaml
    params = 'Params.yaml'
    settings = {'label': 1, 'correctMask': True}
    extractor = featureextractor.RadiomicsFeatureExtractor(params, additionalInfo=True, **settings)

    # Get x_train and threshold images
    x_train = [images_data[key][0] for key in images_data]
    x_train_otsu = [images_data[key][1] for key in images_data]
    x_train_adapt = [images_data[key][2] for key in images_data]

    # Extract features from Otsu's and adaptative
    feats_o, feats_a = run_extractor(x_train, x_train_otsu, x_train_adapt, extractor)

    # Process features and get feature names
    all_feats_o, all_feats_a, names = process_features(feats_o, feats_a)

    # Save features in the dictionary
    keys = list(images_data.keys())
    for i in range(len(images_data)):
        key = key[i]
        value = images_data[key]
        images_data[key] = (value[0], value[1], value[2], all_feats_o[i], all_feats_a[i])

    return (images_data, names)

In [102]:
images_data, names = extract_features(folds)

--2023-11-22 13:21:35--  https://raw.githubusercontent.com/AIM-Harvard/pyradiomics/master/examples/exampleSettings/Params.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3037 (3.0K) [text/plain]
Saving to: ‘Params.yaml’

Params.yaml         100%[===================>]   2.97K  --.-KB/s    in 0s      

2023-11-22 13:21:35 (45.9 MB/s) - ‘Params.yaml’ saved [3037/3037]



INFO:radiomics.featureextractor:Loading parameter file Params.yaml
INFO:radiomics.featureextractor:Applying custom setting overrides: {'additionalInfo': True, 'label': 1, 'correctMask': True}


TypeError: ignored

In [ ]:
import csv
for i in range(len(all_folds_feats)):
    feat_o = all_folds_feats[i][0]
    feat_a = all_folds_feats[i][1]
    filename_o = f'fold{i+1}_o.csv'

    filename_a = f'fold{i+1}_a.csv'

    with open(filename_o, 'w', newline='') as f:
    # Step 4: Using csv.writer to write the list to the CSV file
        writer = csv.writer(f)
        writer.writerows(feat_o) # Use writerow for single list

    with open(filename_a, 'w', newline='') as f:
    # Step 4: Using csv.writer to write the list to the CSV file
        writer = csv.writer(f)
        writer.writerows(feat_a) # Use writerow for single list

In [ ]:
with open('names.txt', 'w') as f:
    f.write(names)

In [ ]:
len(all_folds_feats[1][0][0])


129